In [2]:
from pybis import Openbis
import json
import ipywidgets as widgets
from IPython.display import display, Javascript, clear_output, HTML

In [3]:
# Global variables
OPENBIS_SESSION = None
SAMPLES_COLLECTION_OPENBIS_PATH = "/MATERIALS/SAMPLES/SAMPLE_COLLECTION"

In [5]:
# Widgets
materials_dropdown = widgets.Dropdown(description='Crystal', disabled=False, layout = widgets.Layout(width = '350px'))
materials_dropdown.style = {'description_width': '100px'}
material_details_textbox = widgets.Textarea(value = '', description = '', disabled = True, layout = widgets.Layout(width = '425px', height = '200px'))
material_image_box = widgets.Image(value = open("images/white_screen.jpg", "rb").read(), format = 'jpg', width = '200px', height = '300px', layout=widgets.Layout(border='solid 1px #cccccc'))
material_metadata_boxes = widgets.HBox([materials_dropdown, material_details_textbox, material_image_box])

material_selection_radio_button = widgets.RadioButtons(description = 'Material', options=['No material', 'Crystal', 'Wafer substrate', '2D-layer material'], disabled = False, layout = widgets.Layout(width = '300px'))
material_selection_radio_button.style = {'description_width': '100px'}

sample_name_textbox = widgets.Text(value = '', placeholder = 'Enter sample name', description = 'Name', disabled = False, layout = widgets.Layout(width = '300px'))
sample_name_textbox.style = {'description_width': '100px'}

create_sample_button = widgets.Button(description = '', disabled = False, button_style = '', tooltip = 'Save sample', icon = 'save', layout = widgets.Layout(width = '100px', height = '50px'))
quit_button = widgets.Button(description = '', disabled = False, button_style = '', tooltip = 'Main menu', icon = 'home', layout = widgets.Layout(width = '100px', height = '50px'))
increase_buttons_size = HTML("""
<style>
    .fa-save {
        font-size: 2em !important; /* Increase icon size */
    }
    .fa-home {
        font-size: 2em !important; /* Increase icon size */
    }
</style>
""")

bottom_buttons_hbox = widgets.HBox([create_sample_button, quit_button])
# bottom_buttons_hbox.layout.margin = '0px 0px 0px 65px'

HTML('''
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/awesomplete/1.1.5/awesomplete.min.css" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/awesomplete/1.1.5/awesomplete.min.js"></script>
''')

In [6]:
# Functions
def read_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Function to close the notebook page
def close_notebook(b):
    display(Javascript('window.location.replace("home.ipynb")'))

# Function to create sample object inside openBIS using information selected in the app
def create_sample_action(b):
    global OPENBIS_SESSION
    
    samples = OPENBIS_SESSION.get_objects(type = "SAMPLE")
    samples_names = [sample.props["$name"] for sample in samples]
    
    if sample_name_textbox.value in samples_names:
        display(Javascript(f"alert('{'Sample name already exists!'}')"))
    else:
        if materials_dropdown.value is None or material_selection_radio_button.value == "No material":
            sample_parents = []
        else:
            sample_parents = [materials_dropdown.value]
        
        OPENBIS_SESSION.new_object(type = "SAMPLE", 
                                collection = SAMPLES_COLLECTION_OPENBIS_PATH,
                                props = {"$name": sample_name_textbox.value},
                                parents = sample_parents).save()
        print("Upload successful!")

# Function to handle changes in the materials dropdown
def load_material_metadata(change):
    global OPENBIS_SESSION
    if materials_dropdown.value is None:
        material_details_textbox.value = ''
        file = open("images/white_screen.jpg", "rb")
        image = file.read()
        material_image_box.value = image
    else:
        if material_selection_radio_button.value == "Crystal":
            property_list = [("Name", "$name"), ("Material", "material"), ("Face", "face"), ("Sample Plate", "sample_plate"), ("Diameter", "diameter"), ("Height", "height"), ("Specifications", "specifications")]
        elif material_selection_radio_button.value == "Wafer substrate":
            property_list = [("Name", "$name"), ("Substrates", "substrates"), ("Sample Plate", "sample_plate"), ("Diameter", "diameter"), ("Height", "height"), ("Thickness", "thickness")]
        elif material_selection_radio_button.value == "2D-layer material":
            property_list = [("Name", "$name"), ("Sample Plate", "sample_plate"), ("2D Layers", "layers_2d"), ("Substrates", "substrates"), ("Width", "width"), ("Length", "length"), ("Thickness", "thickness")]
        
        material_object = OPENBIS_SESSION.get_object(materials_dropdown.value)
        material_dataset = material_object.get_datasets()[0]
        
        if material_dataset is None:
            file = open("images/white_screen.jpg", "rb")
            image = file.read()
            material_image_box.value = image
        else:
            material_dataset.download(destination = "images")
            material_dataset_filenames = material_dataset.file_list
            material_image_filepath = material_dataset_filenames[0]
            file = open(f"images/{material_dataset.permId}/{material_image_filepath}", "rb")
            image = file.read()
            material_image_box.value = image
        
        material_metadata = material_object.props.all()
        material_metadata_string = ""
        for property in property_list:
            if property[1] in ["diameter", "height", "width", "thickness"]:
                if material_metadata[property[1]] is None:
                    material_metadata_string = f"{material_metadata_string} {property[0]}: {material_metadata[property[1]]}\n"
                else:
                    property_dict = json.loads(material_metadata[property[1]])
                    material_metadata_string = f"{material_metadata_string} {property[0]}: {property_dict['value']} {property_dict['unit']}\n"
            else:
                material_metadata_string = f"{material_metadata_string} {property[0]}: {material_metadata[property[1]]}\n"
        material_details_textbox.value = material_metadata_string

# Function to handle changes in the radio button
def select_material_radio_change(change):
    material_details_textbox.value = ''
    
    if material_selection_radio_button.value == "No material":
        clear_output()
        display(material_selection_radio_button)
        
    else: 
        if material_selection_radio_button.value == "Crystal":
            materials = OPENBIS_SESSION.get_objects(type = "CRYSTAL")
            materials_dropdown.description = "Crystal"
            materials_placeholder = "Select crystal..."

        elif material_selection_radio_button.value == "Wafer substrate":
            materials = OPENBIS_SESSION.get_objects(type = "WAFER_SUBSTRATE")
            materials_dropdown.description = "Wafer substrate"
            materials_placeholder = "Select wafer substrate..."
            
        elif material_selection_radio_button.value == "2D-layer material":
            materials = OPENBIS_SESSION.get_objects(type = "2D_LAYERED_MATERIAL")
            materials_dropdown.description = "2D-layer material"
            materials_placeholder = "Select 2D-layer material..."
            
        materials_names_permids = [(f"{material.props['$name']} ({material.permId})", material.permId) for material in materials]
        materials_names_permids.insert(0, (materials_placeholder, None))
        materials_dropdown.options = materials_names_permids
        materials_dropdown.observe(load_material_metadata, names = 'value')
            
        clear_output()
        display(material_selection_radio_button)
        display(material_metadata_boxes)
        
    display(increase_buttons_size)

In [7]:
# Connect with openBIS
session_data = read_json("token.json")
OPENBIS_SESSION = Openbis(session_data["url"], verify_certificates = False)
OPENBIS_SESSION.set_token(session_data["token"])

# Create a sample object in openBIS

## Select material

In [ ]:
material_selection_radio_button.observe(select_material_radio_change, names='value')
display(material_selection_radio_button)

## Sample out

In [ ]:
create_sample_button.on_click(create_sample_action)
quit_button.on_click(close_notebook)
sample_name_textbox.add_class('autocomplete-textbox')
display(sample_name_textbox, bottom_buttons_hbox)
display(increase_buttons_size)